In [5]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import KFold

In [6]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import Sampler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "nci"  # Dataset{gdsc or ccle}


args = Args()
res, drug_feature, exprs, mut, cna, null_mask, pos_num = load_data(args)

load nci


In [8]:
epochs = []
true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)
device = "cpu"

for train_index, test_index in kfold.split(np.arange(pos_num)):
    sampler = Sampler(res, train_index, test_index, null_mask)
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    epochs.append(epoch)
    true_datas = pd.concat([true_datas, translate_result(true_data)])
    predict_datas = pd.concat([predict_datas, translate_result(predict_data)])

epoch:   0 loss:0.806117 auc:0.4978
epoch:  20 loss:0.574866 auc:0.7010
epoch:  40 loss:0.467507 auc:0.7480
epoch:  60 loss:0.386741 auc:0.7912
epoch:  80 loss:0.346705 auc:0.8096
epoch: 100 loss:0.326630 auc:0.8193
epoch: 120 loss:0.315510 auc:0.8250
epoch: 140 loss:0.308773 auc:0.8284
epoch: 160 loss:0.304357 auc:0.8305
epoch: 180 loss:0.301271 auc:0.8318
epoch: 200 loss:0.298994 auc:0.8328
epoch: 220 loss:0.297257 auc:0.8336
epoch: 240 loss:0.295905 auc:0.8343
epoch: 260 loss:0.294838 auc:0.8350
epoch: 280 loss:0.293997 auc:0.8354
epoch: 300 loss:0.293333 auc:0.8358
epoch: 320 loss:0.292807 auc:0.8360
epoch: 340 loss:0.292387 auc:0.8361
epoch: 360 loss:0.292047 auc:0.8362
epoch: 380 loss:0.291767 auc:0.8363
epoch: 400 loss:0.291534 auc:0.8364
epoch: 420 loss:0.291335 auc:0.8364
epoch: 440 loss:0.291165 auc:0.8365
epoch: 460 loss:0.291017 auc:0.8366
epoch: 480 loss:0.290888 auc:0.8366
epoch: 500 loss:0.290775 auc:0.8367
epoch: 520 loss:0.290675 auc:0.8368
epoch: 540 loss:0.290587 auc

In [9]:
true_datas.reset_index(drop=True).to_csv("true_nci.csv")
predict_datas.reset_index(drop=True).to_csv("pred_nci.csv")